In [4]:
# reading in packages
import pandas as pd
from mapbox import Geocoder
import numpy as np
import requests
import json
from math import isnan

In [5]:
# Reading in Locations Dataset
vic_markets = pd.read_csv('tmpKqjfH4markets.csv')

# developer access token
MAPBOX_ACCESS_TOKEN = 'pk.eyJ1IjoiZGFzMDAwMSIsImEiOiJjazl3bHVxMzgwYXQ4M2lta2JoYmRraTZ4In0.0wctA_QOgf_1eIdZzmfcHA'
# Create full address
#vic_markets['Full Address'] = vic_markets['Name'] + ' ' + vic_markets['Address'] + ' ' + vic_markets['Suburb']
# Instantiate Geocoder with the access token
geocoder = Geocoder(access_token=MAPBOX_ACCESS_TOKEN)

In [ ]:
lat = []
lon = []
error = []

for row in vic_markets.itertuples():
    try:
        response = geocoder.forward(str(row['Full Address']), country=['au'], limit = 1) # call mapbox
        coordinates = response.json()['features'][0]['geometry']['coordinates'] # get coordinates
        lat.append(coordinates[1]) # store latitude
        lon.append(coordinates[0]) # store coordinates
    except:
        try: # if you get an error, try with the name instead
            response = geocoder.forward(str(row.Name), country=['au'], limit = 1) # call mapbox
            coordinates = response.json()['features'][0]['geometry']['coordinates'] # get coordinates
            lat.append(coordinates[1]) # store latitude
            lon.append(coordinates[0]) # store coordinates
        except:
            try: # if that fails, try with postcode
                response = geocoder.forward(str(int(row.Postcode)), country=['au'], limit = 1) # call mapbox
                coordinates = response.json()['features'][0]['geometry']['coordinates'] # get coordinates
                lat.append(coordinates[1]) # store latitude
                lon.append(coordinates[0]) # store coordinates
            except:
                print("Error!")
                error.append(row)

In [ ]:
# store into dataframe
vic_markets['lat'] = lat
vic_markets['lon'] = lon

# Attempt to do a JOIN on postcodes dataset

In [6]:
postcodes_csv = pd.read_csv('australian_postcodes.csv')
postcodes_csv.drop_duplicates(subset=['postcode'], inplace = True) # get rid of duplicate postcodes, we just want to know 1 lat long

result = pd.merge(vic_markets, postcodes_csv[['postcode', 'long', 'lat']], left_on = ['Postcode'], right_on = ['postcode'], how = 'left')
result.drop(['postcode'], axis = 1, inplace = True)

In [7]:
# Use mapbox for the outliers ;)
lat = []
lon = []
error = []

for row in result[np.isnan(result['Postcode'])].itertuples():
    try:
        response = geocoder.forward(str(row.Name), country=['au'], limit = 1) # call mapbox
        coordinates = response.json()['features'][0]['geometry']['coordinates'] # get coordinates
        lat.append(coordinates[1]) # store latitude
        lon.append(coordinates[0]) # store coordinates
    except:
        try: # if you get an error, try with the name instead
            response = geocoder.forward(str(row.Address), country=['au'], limit = 1) # call mapbox
            coordinates = response.json()['features'][0]['geometry']['coordinates'] # get coordinates
            lat.append(coordinates[1]) # store latitude
            lon.append(coordinates[0]) # store coordinates
        except:
            try: # if that fails, try with postcode
                response = geocoder.forward(str(row.Suburb), country=['au'], limit = 1) # call mapbox
                coordinates = response.json()['features'][0]['geometry']['coordinates'] # get coordinates
                lat.append(coordinates[1]) # store latitude
                lon.append(coordinates[0]) # store coordinates
            except:
                print("Error!")
                error.append(row)

result.loc[np.isnan(result['Postcode']),'lat'] = lat
result.loc[np.isnan(result['Postcode']),'long'] = lon

In [8]:
result.to_csv('vic_markets.csv')

# Uploading to API
Now that we have latitude and longitude for all locations, we should upload this to our Cosmos DB (using our API so that the locations are available)

In [9]:
# cleanup result
result.rename(columns={'Business Category':'business_category'}, inplace=True)
result = result.where(pd.notnull(result), None)

In [10]:
# format a row into the format required for the database
result.iloc[0]

Name                 Ararat Seasonal Farmers' Market
Address                             Lakeside Gardens
Suburb                                        Ararat
Postcode                                        3377
State                                            VIC
business_category                  Fresh Food Market
LGA                                      Ararat (RC)
Region                              Grampians Region
long                                         142.957
lat                                         -37.2315
Name: 0, dtype: object

In [11]:
# iterate on each row in the addresses dataset
error = []
for row in result.itertuples():
    # extract variables
    name = row.Name
    address = row.Address
    suburb = row.Suburb
    state = row.State
    business_category = row.business_category
    lga = row.LGA
    region = row.Region
    lon = row.long
    lat = row.lat
    
    # handle the troublesome postcode
    try:
        postcode = int(row.Postcode)
    except:
        postcode = row.Postcode
    
    # form the payload
    market = {
        "name": name,
        "address": address,
        "suburb": suburb ,
        "postcode": postcode,
        "state": state ,
        "business_category": business_category,
        "lga": lga,
        "region": region,
        "lon": lon,
        "lat": lat,
      }
    
    # making the API call
    r = requests.post('https://gfood-api.azurewebsites.net/markets/add', json=market)
    try:
        if r.status_code != 201:
            error.append(row)
            print("ERROR!")
            print(r.json())
        else:
            print(r.json())
    except:
        continue

{'message': 'Market Added', 'addedMarket': {'name': "Ararat Seasonal Farmers' Market", '_id': '5eb7cab9380e65004468f3fe', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cab9380e65004468f3fe'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Bairnsdale Farmers Market', '_id': '5eb7cabc380e65004468f3ff', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cabc380e65004468f3ff'}}}
{'message': 'Market Added', 'addedMarket': {'name': "Ballarat Lakeside Farmers' Market", '_id': '5eb7cabc380e65004468f400', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cabc380e65004468f400'}}}
{'message': 'Market Added', 'addedMarket': {'name': "Bendigo Farmers' Market", '_id': '5eb7cabd380e65004468f401', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cabd380e65004468f401'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Big Sams St Albans Market', '_id': '5eb

{'message': 'Market Added', 'addedMarket': {'name': 'Templestowe Rotary Farmers Produce Market', '_id': '5eb7cad2380e65004468f423', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cad2380e65004468f423'}}}
{'message': 'Market Added', 'addedMarket': {'name': "Tolmie Farmers' Market", '_id': '5eb7cad3380e65004468f424', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cad3380e65004468f424'}}}
{'message': 'Market Added', 'addedMarket': {'name': "Traralgon Farmers' Market", '_id': '5eb7cad5380e65004468f425', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cad5380e65004468f425'}}}
{'message': 'Market Added', 'addedMarket': {'name': "Veg Out St Kilda Farmers' Market", '_id': '5eb7cad5380e65004468f426', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cad5380e65004468f426'}}}
{'message': 'Market Added', 'addedMarket': {'name': "Whitehorse Farmers' Market", '_

{'message': 'Market Added', 'addedMarket': {'name': 'Beaufort Market', '_id': '5eb7cae6380e65004468f448', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cae6380e65004468f448'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Beechworth Country Craft Market', '_id': '5eb7cae7380e65004468f449', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cae7380e65004468f449'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Belmont Market', '_id': '5eb7cae7380e65004468f44a', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cae7380e65004468f44a'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Benalla Lakeside Quality Craft Market', '_id': '5eb7cae8380e65004468f44b', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cae8380e65004468f44b'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Bendigo Market', '_id': '5eb7cae8380e65004468f44c', 'r

{'message': 'Market Added', 'addedMarket': {'name': 'Chelsea Trash & Treasure Craft Market', '_id': '5eb7cafa380e65004468f46d', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cafa380e65004468f46d'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Chelsea Trash & Treasure & Craft Market', '_id': '5eb7cafb380e65004468f46e', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cafb380e65004468f46e'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Cheltenham Rotary Market & Car Boot Sale', '_id': '5eb7cafb380e65004468f46f', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cafb380e65004468f46f'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Christmas Hills Community Market & Car Boot Sale', '_id': '5eb7cafd380e65004468f470', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cafd380e65004468f470'}}}
{'message': 'Market Added', 'addedMarket

{'message': 'Market Added', 'addedMarket': {'name': 'Grantville Variety Market', '_id': '5eb7cb0f380e65004468f492', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb0f380e65004468f492'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Greensborough Community Market', '_id': '5eb7cb0f380e65004468f493', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb0f380e65004468f493'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Gully Market', '_id': '5eb7cb0f380e65004468f494', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb0f380e65004468f494'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Gumbuya Park Monthly Craft Market', '_id': '5eb7cb10380e65004468f495', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb10380e65004468f495'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Halls Gap Art & Craft Market', '_id': '5eb7cb10380e

{'message': 'Market Added', 'addedMarket': {'name': 'Longwarry Market', '_id': '5eb7cb21380e65004468f4b7', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb21380e65004468f4b7'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Lucknow Primary School Sunday Market', '_id': '5eb7cb21380e65004468f4b8', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb21380e65004468f4b8'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Mansfield Bush Market', '_id': '5eb7cb22380e65004468f4b9', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb22380e65004468f4b9'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Maryborough Sunday Tourist Market', '_id': '5eb7cb22380e65004468f4ba', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb22380e65004468f4ba'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Marysville Community Market', '_id': '5eb7cb2

{'message': 'Market Added', 'addedMarket': {'name': 'Point Cook Community Market', '_id': '5eb7cb33380e65004468f4dc', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb33380e65004468f4dc'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Point Lonsdale Primary School Market', '_id': '5eb7cb34380e65004468f4dd', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb34380e65004468f4dd'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Pomonal Village Market', '_id': '5eb7cb34380e65004468f4de', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb34380e65004468f4de'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Port Albert Market', '_id': '5eb7cb35380e65004468f4df', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb35380e65004468f4df'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Port Campbell Summer Market', '_id': '5eb7cb3538

{'message': 'Market Added', 'addedMarket': {'name': 'The Campus Monash University Market', '_id': '5eb7cb46380e65004468f501', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb46380e65004468f501'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'The Gardens Market', '_id': '5eb7cb47380e65004468f502', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb47380e65004468f502'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'The Makers Bazaar', '_id': '5eb7cb47380e65004468f503', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb47380e65004468f503'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'The Market at Yarrawood', '_id': '5eb7cb48380e65004468f504', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb48380e65004468f504'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'The Sunday Market', '_id': '5eb7cb48380e65004468f505', 're

{'message': 'Market Added', 'addedMarket': {'name': 'Yarra Junction Football Club Market', '_id': '5eb7cb59380e65004468f526', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb59380e65004468f526'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Yarram Market', '_id': '5eb7cb59380e65004468f527', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb59380e65004468f527'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Yarrawonga Rotary Market', '_id': '5eb7cb5a380e65004468f528', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb5a380e65004468f528'}}}
{'message': 'Market Added', 'addedMarket': {'name': 'Yea Community Craft Market', '_id': '5eb7cb5a380e65004468f529', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/markets/5eb7cb5a380e65004468f529'}}}


In [12]:
error

[Pandas(Index=184, Name='Loch Village Market', Address='Loch Village Railway Siding', Suburb='Loch (15km west of Korumburra)', Postcode=None, State=None, business_category='Market', LGA=None, Region=None, long=145.7067, lat=-38.3697)]